#### Objective:

Create a chatbot to check credit card balance

In [1]:
#import the Python packages for Lambda to use
import boto3
from boto3.dynamodb.conditions import Key, Attr
import json
import decimal

In [2]:
#start our Lambda runtime here 
def lambda_handler(event_data, lambda_config):

    
    #Retrieve ANI from inbound callerID
    #callerId = event_data["sessionAttributes"]["CustomerNumber"]
    callerId = "+60176043930"
    
    #Lex Intent name
    intent_name = event_data['currentIntent']['name']
    
    #Lex session attributes
    session_attributes = event_data['sessionAttributes']
    #print("Here # 1")
    
    #Establish connection to dynamoDB and retrieve table
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('RICHA_ASHISH_DEV')
    client = boto3.client('lex-models')
    
    #Establish connection to dynamoDB and retrieve table
    #sns_client = boto3.client('sns')
    
    #KeyConditionExpression looks for number that equals ANI of inbound call from a dynamoDB table and saves it to response
    response = table.query(
        KeyConditionExpression=Key('phone_number').eq(callerId)
        )
        
    #print("Here # 2")    
    #will always return list
    items = response.get('Items')
    #cust_acct_type = event_data["currentIntent"]["slots"]["accType"]
    #cust_acct_type = event_data["currentIntent"]["slots"]["typeOfAccount"]
    #is_acct_balance = event_data["currentIntent"]["slots"]["isAccountBalance"]
    cust_acct_type = event_data["currentIntent"]["slots"]["credAccType"]
    
    cred_1_due_amount = items[0].get('cc_1_due_amount')
    cred_1_due_amount = decimal.Decimal(cred_1_due_amount)
    print(cred_1_due_amount)
    
    cred_1_due_date = items[0].get('cc_1_due_date')
    cred_1_due_date = decimal.Decimal(cred_1_due_date)
    print(cred_1_due_date)
    
    cred_2_due_amount = items[0].get('cc_2_due_amount')
    cred_2_due_amount = decimal.Decimal(cred_2_due_amount)
    print(cred_2_due_amount)
    
    cred_2_due_date = items[0].get('cc_2_due_date')
    cred_2_due_date = decimal.Decimal(cred_2_due_date)
    print(cred_2_due_date)
    
    if not items:
        return {'acct_balance_1' : 0, "acct_type": "not available"}
    else:
        
        if(intent_name == "checkCreditCardBal"):
            
            if (cust_acct_type in ("platinum")):
                acct_balance = cred_1_due_amount
                message = 'You have ' + str(acct_balance) + ' ringgit left on your ' + cust_acct_type + ' account.'
                #message = 'You have ' + str(cred_1_due_date) + ' days left to settle bills for your ' + cust_acct_type + ' account.'
                #return {'acct_balance' : acct_balance, "cust_acct_type": cust_acct_type}
                return close(session_attributes, "Fulfilled", '<speak>{}</speak>'.format(message))
                
            elif (cust_acct_type in ("signature")):
                acct_balance = cred_2_due_amount
                message = 'You have ' + str(acct_balance) + ' ringgit left on your ' + cust_acct_type + ' account.'
                #message = 'You have ' + str(cred_2_due_date) + ' days left to settle bills for your ' + cust_acct_type + ' account.'
                #return {'acct_balance' : acct_balance, "cust_acct_type": cust_acct_type}
                return close(session_attributes, "Fulfilled", '<speak>{}</speak>'.format(message))
            
def close(session_attributes, fulfillment_state, message):
    response = {
    'sessionAttributes': session_attributes,
        "dialogAction": {
            "type": "Close",
            "fulfillmentState": "Fulfilled", 
            "message": {
                "contentType": "SSML",
                "content": message
                }
            }
        }
    return response
    
def build_response_message(message_content):
    return {"contentType": "PlainText", "content": message_content}
    
def elicit_slot(session_attributes, intent_name, slots, slot_to_elicit, message):
    return {
        'sessionAttributes': session_attributes,
        'dialogAction': {
            'type': 'ElicitSlot',
            'intentName': intent_name,
            'slots': slots,
            'slotToElicit': slot_to_elicit,
            'message': {
                "contentType": "SSML",
                "content": message
                }
            }
        }